In [1]:
from typing import List

from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.agent import AgentMessage
from aap_core.orchestration import ReflectionAgent
from aap_core.types import AgentResponse
from aap_dspy.chain import BaseSignatureAdapter, ChatCausalMultiTurnsChain

import dspy

In [2]:
from typing import Dict


class QA(dspy.Signature):
    guide: str = dspy.InputField(optional=True)
    response: str = dspy.InputField(optional=True)
    question: str = dspy.InputField()
    answer: str = dspy.OutputField()
    history: dspy.History = dspy.InputField()


class SignatureAdapter(BaseSignatureAdapter[QA]):
    _signature_map: Dict[str, str]

    @classmethod
    def with_mapping(cls, prefill_dict: Dict[str, str], context_map: Dict[str, str]):
        obj = cls()
        obj._prefill_dict = prefill_dict
        for v in context_map.values():
            if v not in QA.fields.keys():
                raise ValueError(f"Invalid context key: {v}")
        obj._signature_map = context_map
        return obj

    def msg2sig(self, message: AgentMessage) -> List[QA]:
        history = dspy.History(messages=[])
        q, a = None, None
        for response in message.responses:
            if response[0] == "user":
                q = response[1]
            elif response[0] == "tool":
                pass
            elif response[0] == "system":
                pass
            else:
                a = response[1]

            if q is not None and a is not None:
                history.messages.append({"question": q, "answer": a})
                q, a = None, None

        sig_dict = {}
        for k, v in self._signature_map.items():
            if k.startswith("context.") and message.context is not None:
                sig_dict[v] = message.context.get(k.replace("context.", ""), "")
            elif k == "query":
                sig_dict[v] = message.query
        for k in QA.output_fields:
            sig_dict[k] = ""
        for k, v in QA.input_fields.items():
            if k not in sig_dict and v.annotation is not dspy.History:
                sig_dict[k] = ""
        sig_dict["guide"] = self._prefill_dict.get("guide", "")
        qa = QA(history=history, **sig_dict)
        return [qa]

    def sig2msg(self, signatures: List[QA], name: str) -> List[AgentResponse]:
        responses = []
        for signature in signatures:
            # responses.append(("user", signature.question))
            responses.append((name, signature.answer))
        return responses

In [3]:
model = dspy.LM(model="ollama_chat/gemma3:12b-it-qat", api_base="http://192.168.55.1:11434", api_key="", cache=False, max_tokens=32768)

In [ ]:
# Notice that with dspy, we don't have user prompt template, as the signature is doing the job already
predictor = dspy.ChainOfThought(QA)

system_prompt_task = """/no_think You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Some requirements:
- The logic is clear and easy to understand.
- The function arguments and return values (if any) should be typed.
- If the function is too long (for example greater than 80 lines), split the logic into multiple smaller functions.
- All functions should have docstring explanation. In the explanation, there should be an simple example to illustrate the function and how to call it.
- The response should contain function with docstring explanation. And DO NOT contain explanation outside of the code
"""
task_adapter = SignatureAdapter.with_mapping(prefill_dict={"guide": system_prompt_task}, context_map={"query": "question"})
task_chain = ChatCausalMultiTurnsChain(QA, predictor=predictor, adapter=task_adapter).with_lm(model)
task_chain.final_response_as_context("response")

system_prompt_reflection = """/no_think You are a excellent code reviewer and refactor.
Given a function implementation and it explanation, your task is to review and code and correct if contains any mistake.
Some note:
- For the implementation, check if the orignal query and suggested implementation are match.
- Is there any syntax error in the code.
- For the explanation, verify if the docstring follows Google style docstring.
- In the docstring, make sure to have an example to call the function.
- Make sure the final output only contain full function code, inline code comment and docstring, nothing else.
- If no modificatoin is made, just return the original implementation"""
# Use the same llm for task and self-reflection
reflection_adapter = SignatureAdapter.with_mapping(prefill_dict={"guide": system_prompt_reflection}, context_map={"query": "question", "context.response": "response"})
reflection_chain = ChatCausalMultiTurnsChain(QA, predictor=predictor, adapter=reflection_adapter).with_lm(model)  # use the same model for self-reflection

def state_callback(state: str):
    print(f"agent state: {state}")
reflection_skill = AgentSkill(
    id='reflection-skill',
    name="reflection skill",
    description="self-reflection skill",
    tags=['reflection']
)
reflection_card = AgentCard(
    name="reflection agent",
    description="self-reflection agent",
    skills=[reflection_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
reflection_agent = ReflectionAgent(card=reflection_card, chain_task=task_chain, chain_reflection=reflection_chain, state_change_callback=state_callback)

In [5]:
# Take a leetcode as an example. Source: https://leetcode.com/problems/palindrome-partitioning-ii/description/
query = """Write python function(s) to solve the following problem:
Given a string s, partition s such that every substring of the partition is a palindrome.
Return the minimum cuts needed for a palindrome partitioning of s.

Example 1:
Input: s = "aab"
Output: 1
Explanation: The palindrome partitioning ["aa","b"] could be produced using 1 cut.

Example 2:
Input: s = "a"
Output: 0

Example 3:
Input: s = "ab"
Output: 1

Constraints:
1 <= s.length <= 2000
s consists of lowercase English letters only."""

final_message = reflection_agent.execute(AgentMessage(query=query))
for response in final_message.responses:
    name, msg = response
    print(name)
    print(msg)

agent state: reflection agent:running
agent state: reflection agent:reflecting
agent state: reflection agent:idle
reflection agent
```python
def is_palindrome(s: str) -> bool:
    """
    Checks if a string is a palindrome.

    Example:
    is_palindrome("aba") == True
    is_palindrome("abc") == False
    """
    return s == s[::-1]

def min_palindrome_cuts(s: str) -> int:
    """
    Calculates the minimum cuts needed to partition a string into palindromic substrings.

    Example:
    min_palindrome_cuts("aab") == 1
    min_palindrome_cuts("a") == 0
    min_palindrome_cuts("ab") == 1
    """
    n = len(s)
    dp = [i for i in range(n)]

    for i in range(1, n):
        if s[:i+1] == s[:i+1][::-1]:
            dp[i] = 0
        else:
            for j in range(i):
                if s[j+1:i+1] == s[j+1:i+1][::-1]:
                    dp[i] = min(dp[i], dp[j] + 1)
    return dp[n-1]
```
reflection agent
```python
def is_palindrome(s: str) -> bool:
    """
    Checks if a string is 

In [7]:
dspy.inspect_history(n=10)





[2025-12-11T17:23:12.080639]

System message:

Your input fields are:
1. `guide` (str): 
2. `response` (str): 
3. `question` (str): 
4. `history` (History):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## guide ## ]]
{guide}

[[ ## response ## ]]
{response}

[[ ## question ## ]]
{question}

[[ ## history ## ]]
{history}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `guide`, `response`, `question`, `history`, produce the fields `answer`.


User message:

[[ ## guide ## ]]
/no_think You are a helpful coding assistant.
You task is to write a python function and return the implementation of the function.
Some requirements:
- The logic is clear and easy to understand.
- The function arguments and return values (if any) should be typed.
- If the functio